In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install rouge_score
!pip install transformers
!pip install datasets
!pip install parlai
!pip install transformers datasets wandb
!pip install --upgrade accelerate
!parlai display_data -t msc:PersonaSummary --include-last-session True
!pip install --upgrade numpy
!pip install names
!pip install py7zr


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.0
    Uninstalling numpy-1.25.0:
      Successfully uninstalled numpy-1.25.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-19 22:10:58.705476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 22:10:59.682433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-19 22:11:01.095738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-19 22:11:01.096242

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
max_input = 512
max_target = 128
batch_size = 3
model_checkpoint = "philschmid/flan-t5-base-samsum"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
import names
#Generate random names
for i in range(10):
    rand_name = names.get_first_name()
    print(rand_name)

Mary
Linda
Kathleen
William
Bobbie
Jennifer
Judith
Raymond
Walter
Christopher


In [7]:
from datasets import load_dataset, load_from_disk

In [8]:
# data = load_dataset("samsum")

In [9]:
def getSpeakerNames():
  speaker_1 = names.get_first_name()
  speaker_2 = names.get_first_name()
  while(speaker_1 == speaker_2):
    speaker_2 = names.get_first_name()
  return speaker_1, speaker_2

In [10]:
import re
import json
# Read the JSON file
folder_path = '/usr/local/lib/python3.10/dist-packages/data/msc/msc/msc_personasummary'

def createDataset(mode):
    s1_data_path = f"{folder_path}/session_1/{mode}.txt"
    s2_data_path = f"{folder_path}/session_2/{mode}.txt"
    s3_data_path = f"{folder_path}/session_3/{mode}.txt"
    s4_data_path = f"{folder_path}/session_4/{mode}.txt"

    if mode == 'train':
        data_paths = [s1_data_path, s2_data_path, s3_data_path]
    else:
        data_paths = [s1_data_path, s2_data_path, s3_data_path, s4_data_path]

    mode_X = []
    mode_y = []
    # 데이터 파일 열기

    for data_path in data_paths:
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                # 한 줄씩 읽기
                data = json.loads(line)
                train_data = ""
                name1, name2 = getSpeakerNames()
                count = 0
                id0_prev_count = 0
                id1_prev_count = 0
                labels = []
                for idx, utterance in enumerate(data['dialog']):
                    count += 1
                    if utterance['id'] != 'bot_0' and utterance['id'] != 'bot_1': assert(0)
                    bot_id = utterance['id']
                    speaker_name = name1 if utterance['id'] == 'bot_0' else name2
                    text = utterance['text']
                    agg_persona_list = utterance['agg_persona_list']

                    if bot_id == 'bot_0':
                        summary = agg_persona_list[id0_prev_count:]
                    else:
                        summary = agg_persona_list[id1_prev_count:]
                    for i in range(len(summary)):
                        summary[i] = summary[i].replace('I', speaker_name)
                        summary[i] = summary[i].replace("'ve", "'s")
                        summary[i] = summary[i].replace("My", f"{speaker_name}'s")
                        summary[i] = summary[i].replace("my", f"{speaker_name}'s")
                        summary[i] = summary[i].replace("am", "is")
                        summary[i] = summary[i].replace("'m", " is")

                    if bot_id == 'bot_0':
                        id0_prev_count += len(summary)
                    else:
                        id1_prev_count += len(summary)
                    # 이어붙이기
                    train_data += f"{speaker_name}: {text}\r\n"
                    labels.extend(summary)
                    if count == 4:
                        train_data = train_data.rstrip('\r\n')
                        summary_list = list(dict.fromkeys(labels))
                        joined_summaries = " ".join(summary_list)
                        # print(train_data)
                        # print(joined_summaries)
                        mode_X.append(train_data)
                        mode_y.append(joined_summaries)
                        train_data = ""
                        labels = []
                        count = 0
    return mode_X, mode_y

In [11]:
train_X, train_y = createDataset(mode='train')
valid_X, valid_y = createDataset(mode='valid')

In [12]:
def viewData(mode, idx):
    if mode == 'train':
        text = train_X[idx]
        summary = train_y[idx]
    else:
        text = valid_X[idx]
        summary = valid_y[idx]
    text_list = text.split('\r\n')
    summary_list = summary.split('. ')
    print('<Text>')
    for t in text_list:
        print(t+'\n')
    print('<Summary>')
    for s in summary_list:
        print(s+'\n')

In [13]:
viewData('train', 4)

<Text>
Emily: Sounds good. What kind will you get?

Kim: A pit bull. I'll get him when I find a teaching job

Emily: I teach drums and I play them really well.

Kim: Wow my brother is in a heavy medal group and goes all around the world

<Summary>
Kim plan on getting a job in teaching

Emily play the drums.



In [14]:
from datasets import Dataset

train_dataset = Dataset.from_dict({'text':train_X, 'label':train_y})
valid_dataset = Dataset.from_dict({'text':valid_X, 'label':valid_y})

In [15]:
print('Train Dataset Length : ', len(train_X))
print('Valid Dataset Length : ', len(valid_X))

Train Dataset Length :  32320
Valid Dataset Length :  6251


In [16]:
prefix = 'summarize: '
def preprocess_data(sample):
    text = prefix + sample['text']
    #tokenize the dialogues
    model_inputs = tokenizer(text,  max_length=max_input, padding='max_length', truncation=True)
    #tokenize the summaries
    with tokenizer.as_target_tokenizer():
      targets = tokenizer(sample['label'], max_length=max_target, padding='max_length', truncation=True)

    #set labels
    model_inputs['labels'] = targets['input_ids']
    #return the tokenized data
    #input_ids, attention_mask and labels
    return model_inputs

In [17]:
tokenized_train_dataset = train_dataset.map(preprocess_data)
tokenized_valid_dataset = valid_dataset.map(preprocess_data)

  0%|          | 0/32320 [00:00<?, ?ex/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/6251 [00:00<?, ?ex/s]

In [18]:
tokenized_train_dataset[2]

{'text': 'Kelly: Kittens really? I rather be at the beach.\r\nRobert: Only if the beach was covered in kittens!\r\nKelly: That would be a sight to see.\r\nRobert: Or maybe brownies... I love chocolate.',
 'label': 'Kelly like going to the beach. Robert love chocolate.',
 'input_ids': [21603,
  10,
  10605,
  10,
  5747,
  324,
  7,
  310,
  58,
  27,
  1066,
  36,
  44,
  8,
  2608,
  5,
  2715,
  10,
  3462,
  3,
  99,
  8,
  2608,
  47,
  2303,
  16,
  25573,
  7,
  55,
  10605,
  10,
  466,
  133,
  36,
  3,
  9,
  6398,
  12,
  217,
  5,
  2715,
  10,
  955,
  2087,
  28427,
  7,
  233,
  27,
  333,
  3711,
  5,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [19]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['text', 'label'])
tokenized_valid_dataset = tokenized_valid_dataset.remove_columns(['text', 'label'])

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [21]:
batch_size = 8
model_name = "Flan-T5_4chats"
model_dir = f"drive/MyDrive/Colab Notebooks/Metabuddy/Models/week3/{model_name}"
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=100,
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=False,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to='wandb',
    push_to_hub=False,
    logging_dir=f"{model_dir}/runs"
)

In [22]:
label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
    )

In [23]:
from datasets import load_metric

metric = load_metric("rouge")

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
id = wandb.util.generate_id()
print(id)

In [ ]:
# wandb.init(project='Memory Extraction', # 실험기록을 관리한 프로젝트 이름
#            entity='knkim', # 사용자명 또는 팀 이름
#            id='66z160rw',  # 실험에 부여된 고유 아이디
#            name='plan-t5_4chats',    # 실험에 부여한 이름
#           )

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train(resume_from_checkpoint=True)